In [1]:
import numpy as np
from scipy import integrate
import pandas as pd
import time

# 本项目目标为用不同数值方法计算积分并比较其差异
## 我们首先实现各种数值积分算法
### 复化梯形公式

In [2]:
def compound_trapezoidal(f, n, a, b):
    #f为函数，n + 1为求积节点数，[a,b]为积分区间
    h = (b - a) / n   # h为节点间距
    T1 = f(a) + f(b)  # 端点函数值之和
    T2 = np.sum(f(a + h * np.arange(1, n)))   # 内节点函数值之和
    T = h * (T1 + 2 * T2) / 2
    return T

### 复化Simpson公式

In [3]:
def compound_simpson(f, n, a, b):
    #f为函数，n + 1为求积节点数，[a,b]为积分区间，n为偶数
    if n % 2 != 0:
        return "Error!"
    h = (b - a) / n # h为节点间距
    T1 = f(a) + f(b) # 端点函数值之和
    T2 = np.sum(f(a + h * np.arange(2, n, 2)))  # 内偶节点函数值之和
    T3 = np.sum(f(a + h * np.arange(1, n, 2)))  # 内奇节点函数值之和
    T = h * (T1 + 2 * T2 + 4 * T3) / 3
    return T

### Romberg方法

In [4]:
def romberg(f, k0, a, b, eps = 1e-5):
    h = b - a
    # 设k为外推次数，i表示h=(b-a)/2^(i-k)
    T0 = list(np.zeros(k0 + 1)) #T0存储T i-1的积分值
    T1 = list(np.zeros(k0 + 1)) #T1存储T i的积分值
    T0[0] = h / 2 * (f(a) + f(b)) # 初始化时，T0只有1个积分值，i-1=0，h=b-a，即梯形公式值
    T = T0[0] #总是记录上一次外推的结果，如果两次积分结果足够接近说明收敛，eps取0则可使计算持续到k0次外推
    j = 1 # 复化梯形公式内奇节点个数
    for i in range(1, k0 + 1):
        T1[0] = 1 / 2 * (T0[0] + h * sum(f(a + h * np.arange(0.5, j, 1)))) # T1的第一个值k=0，无递推公式，直接使用有1+2^i个节点的复化梯形公式
        # 计算复化梯形公式也使用了逐次分半法中的递推
        for k in range(1, i + 1):# 利用递推式计算i时不同k值的积分值，即计算h=(b-a)/2^(i-k)时k次地推
            T1[k] = (4 ** k * T1[k - 1] - T0[k - 1]) / (4 ** k - 1) # T1[k-1]即T(i,k-1)的值，它的h=(b-a)/2^(i-k+1)，T0[k-1]即T(i-1,k-1)的值，h=(b-a)/2^(i-k)
        if (np.abs(T1[i] - T) < eps): #如果误差很小，则结束运算
            return T1[i]
        h = h / 2 # 为计算下一行做准备，将h减半
        j = j * 2 # 下一次计算复化梯形时，奇数内节点个数翻倍
        for l in range(i + 1):
            T0[l] = T1[l] #用计算出的新的积分值作为T0，这减少了内存占用
        T = T0[i] # 将本次外推结果记录
    return T1[k0]

### Gauss型求积公式

In [5]:
def gauss_3(f, a, b):
    standard_gauss_point = np.array([-0.7745966692, 0, 0.7745966692]) # [-1, 1]上Gauss点
    standard_A = np.array([0.5555555556, 0.8888888889, 0.5555555556]) # [-1, 1]上求积系数
    gauss_point = (b - a) * (standard_gauss_point + 1) / 2 + a # [a, b]上Gauss点
    A = (b - a) * standard_A / 2 # [a, b]上求积系数
    T = np.matmul(A, f(gauss_point))
    return T

def gauss_4(f, a, b):
    standard_gauss_point = np.array([-0.8611363116, -0.3399810436, 0.3399810436, 0.8611363116]) # [-1, 1]上Gauss点
    standard_A = np.array([0.3478548451, 0.6521451549, 0.6521451549, 0.3478548451]) # [-1, 1]上求积系数
    gauss_point = (b - a) * (standard_gauss_point + 1) / 2 + a # [a, b]上Gauss点
    A = (b - a) * standard_A / 2 # [a, b]上求积系数# [a, b]上求积系数
    T = np.matmul(A, f(gauss_point))
    return T

## 定义被积函数

In [6]:
def f(x):
    if not hasattr(x, '__iter__'):
        if x == 0:
            return 1 # 在0处取极限值1
        return np.sin(x) / x
    return np.where(x != 0, np.sin(x) / x, 1)

## 计算数值积分

In [7]:
truth_value = integrate.quad(f, 0, np.pi / 2)[0] # 利用包中的方法计算，视之为真实值，实际误差数量级为10^-14
print("真实值", truth_value)
# 计算各种方法下的积分估计值
result = pd.DataFrame([compound_trapezoidal(f, 8, 0, np.pi / 2),
                       compound_simpson(f, 8, 0, np.pi / 2),
                       romberg(f, 3, 0, np.pi / 2, 0),
                       gauss_3(f, 0, np.pi / 2),
                       gauss_4(f, 0, np.pi / 2)], index = ["复化梯形", "复化Simpson", "Romberg", "3节点Gauss型求积", "4节点Gauss型求积"], columns=["估计值"])

print(result)

真实值 1.3707621681544881
                  估计值
复化梯形         1.369460
复化Simpson    1.370764
Romberg      1.370762
3节点Gauss型求积  1.370763
4节点Gauss型求积  1.370762


## 计算误差和时间

In [8]:
# 计算各种方法的误差
err = np.ones_like(result["估计值"]) * truth_value - result["估计值"]
result.insert(loc=len(result.columns), column='误差值', value=err)

# 通过重复每种算法10000次来计算各种方法所需的时间

t = []
Stime = time.time()
for i in range(10000):
    compound_trapezoidal(f, 8, 0, np.pi / 2)
Etime = time.time()
t.append(Etime - Stime)

Stime = time.time()
for i in range(10000):
    compound_simpson(f, 8, 0, np.pi / 2)
Etime = time.time()
t.append(Etime - Stime)

Stime = time.time()
for i in range(10000):
    romberg(f, 8, 0, np.pi / 2)
Etime = time.time()
t.append(Etime - Stime)

Stime = time.time()
for i in range(10000):
    gauss_3(f, 0, np.pi / 2)
Etime = time.time()
t.append(Etime - Stime)

Stime = time.time()
for i in range(10000):
    gauss_4(f, 0, np.pi / 2)
Etime = time.time()
t.append(Etime - Stime)
t = np.array(t)

result.insert(loc = len(result.columns), column="计算时间*10000", value = t)
print(result)

                  估计值           误差值  计算时间*10000
复化梯形         1.369460  1.302559e-03    0.197472
复化Simpson    1.370764 -1.907924e-06    0.376024
Romberg      1.370762 -9.743051e-10    0.564487
3节点Gauss型求积  1.370763 -1.269137e-06    0.183509
4节点Gauss型求积  1.370762  2.723778e-09    0.182509


In [9]:
# 输出latex公式
print("\\begin{table}\n\\caption{数值结果}\n\\centering")
print(result.to_latex(), "\\end{table}")

\begin{table}
\caption{数值结果}
\centering
\begin{tabular}{lrrr}
\toprule
{} &       估计值 &           误差值 &  计算时间*10000 \\
\midrule
复化梯形        &  1.369460 &  1.302559e-03 &    0.197472 \\
复化Simpson   &  1.370764 & -1.907924e-06 &    0.376024 \\
Romberg     &  1.370762 & -9.743051e-10 &    0.564487 \\
3节点Gauss型求积 &  1.370763 & -1.269137e-06 &    0.183509 \\
4节点Gauss型求积 &  1.370762 &  2.723778e-09 &    0.182509 \\
\bottomrule
\end{tabular}
 \end{table}


C:\Users\APLS\AppData\Local\Temp\ipykernel_17112\39405407.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(result.to_latex(), "\\end{table}")
